In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy as sp
%matplotlib notebook
mylib.rl()

### The purpose of this file is to make the kappa schedule perfectly

In [ ]:
def kappa(a):
        '''This function returns the max-kappa value throughtout the course of an anneal'''
        K = []
        #lets find out the differential of the schedule
        dA = diff(a.ss,a.AB[0])
        dB = diff(a.ss,a.AB[1])
        dAB = np.asarray([dA,dB])
        
        for i in range(a.points):
            # in this loop we calulate the eigen energy gap between (degenerate) ground state and the first excited level
            
            # neeed to be able to find the ground and frist excited states
            d,P,P_in = mylib.Diagonaliser(-a.AB[0][i]*a.Hb+a.AB[1][i]*a.Hp)
            E0 = min(d)
            x0 = P[:,d==E0] #x0 = P[:,d==E0] this line should only work if the ground state is not degenerate
            if len(np.argwhere(d==E0))>1: print '[%s] This is not valid, the ground state was degerate'%i
                
            # we want to consider the kappa value for all states wso that we can consider the maximum one
            pot_k = []
            for i0 in range(len(d)):
                if d[i0]==E0: # don't consider the overlap with itself or any degenerate levels
                    pass
                else:
                    dE = abs(E0-d[i0])
                    xn = P[:,i0] 
                    # here is the real line of calculation. This should be exactly the kappa equation
                    pot_k.append(abs(np.dot(np.transpose(x0),(-dAB[0][i]*a.Hb+dAB[1][i]*a.Hp)*xn).item()/(dE**2)))
            k = max(pot_k)
            K.append(k)
        return np.asarray(K)

def diff(x,y):
    dx0 = np.asarray([x[1]-x[0]]); dy0 = np.asarray([y[1]-y[0]])
    dxf = np.asarray([x[-1]-x[-2]]); dyf = np.asarray([y[-1]-y[-2]])
    dxs = x[2:]-x[:-2]; dys = y[2:]-y[:-2]
    return np.concatenate((dy0/dx0, dys/dxs, dyf/dxf))

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# PREDEFINED CONSTANTS AND RUN ANNEAL
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
T =5
points = 1000
qubits = 6
h_i = np.random.normal(loc=0.0, scale=3.0, size=qubits)
J_ij = np.triu(np.random.normal(loc=0.0, scale=3.0, size=(qubits,qubits)))
ss = np.linspace(0,1,points)

# run the anneal for the constant k schedule and alongside it run the inverse-energy-squared schedule so
# that I can check if the success probability of far from what I expect
a = mylib.Anneal(qubits,[h_i,J_ij], points = points, T = T, sched_func = 'constant_k')
a.run()
a_yardstick = mylib.Anneal(qubits,[h_i,J_ij], points = points, T = T, sched_func = 'inverse_square_energy')
a_yardstick.run()
a.show_results()

print '\n ~~~~~~~~~~~~~~~~~~~~ SUCCESS PROBABILITIES~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n'
print 'Constant k:   %s'%a.problem_x0_prob[-1]
print 'Inv. E^2:    %s'%a_yardstick.problem_x0_prob[-1]

In [ ]:
k = kappa(a)
dP = diff(ss,a.instant_x0s_prob)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# PLOTTING KAPPA AND dP
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
plt.figure(figsize = (13,6))
ax1=plt.subplot2grid((2,4),(0,0),colspan = 3)
ax2=plt.subplot2grid((2,4),(1,0),colspan = 3, sharex = ax1)


# plot the probability of being found in the problem ground state/instanteaous gorund state 
ax1.plot(ss, k)
        
# plot the energy gap between the first 2 levels and the annealing schedule if it is uniform across bits 
ax2.plot(ss,dP)    

ax1.set_ylabel(r'$\kappa (s)$') 
ax2.set_ylabel(r'$\frac{\partial}{\partial s}P(\psi_{0}(s))$',color = 'black')
ax2.set_xlabel('s')
ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,title='$\#$')
ax2.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,title='$Schedule$')
plt.show()

In [ ]:
abs(np.dot(np.transpose(x0),(-dAB[0][0]*a.Hb+dAB[1][0]*a.Hp)*xn).item())